In [ ]:
! pip install selenium

In [2]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")
import time
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import re

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars.
2.In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [4]:
driver = webdriver.Chrome(r"C:\Users\Toshiba\chromedriver.exe")

In [5]:
driver = webdriver.Chrome("chromedriver.exe")
time.sleep(2)

In [6]:
driver.get('https://www.amazon.in/')

In [7]:
inputU = input('please enter product here--->')

please enter product here--->HEADPHONES


In [8]:
search_bar = driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')    # Finding the search bar using it's xpath
search_bar.send_keys(inputU)       # Inputing keyword to search 
search_button = driver.find_element_by_xpath('//*[@id="nav-search-submit-button"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

In [9]:
productName=[]

In [10]:
#scraping the Product_Name 
PName=driver.find_elements_by_xpath("//span[@class='a-size-medium a-color-base a-text-normal']")
for i in PName:
    if i.text is None :
        productName.append("--") 
    else:
        productName.append(i.text)
print(len(productName),productName)

24 ['Boat Rockerz 550 Upto 20 Hours Playback, 50Mm Drivers, Soft Padded Ear Cushions and Physical Noise Isolation Bluetooth Wireless Over Ear Headphones with Mic (Army Green)', 'Tribit XFree Go Headphones with Mic, Wireless Bluetooth Headphone Over Ear, HiFi Sound,Deep Bass,Lightweight,Type-C Lightening Fast Charge, Voice Control,Black', 'Boat Rockerz 450 Bluetooth Wireless On Ear Headphones with Mic, Upto 15 Hours Playback, 40Mm Drivers, Padded Ear Cushions and Dual Modes (Luscious Black)', 'Boat Bassheads 100 Wired in Ear Earphones with Mic (Mint Green)', 'Boat Rockerz 450 Bluetooth Wireless On Ear Headphones with Mic and Upto 15 Hours Playback, 40Mm Drivers, Padded Ear Cushions and Dual Modes (Aqua Blue)', 'Poly by Plantronics - Voyager 4320 UC Wireless Headset (Plantronics) - Headphones with Boom Mic - Connect to PC/Mac via USB-C Bluetooth Adapter, Cell Phone via Bluetooth, Black (218478-02)', 'boAt Rockerz 510 Over Ear Bluetooth Headphones with Upto 20 Hours Playback, 50MM Drivers

In [ ]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scrapped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page

In [12]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Rating']=[]
prod_dict['No. of ratings']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]

In [13]:
for url in urls[:4]:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  # Extracting Ratings from xpath
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    try:
        n_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')     # Extracting no. of Ratings from xpath
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
    
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0721383JGVQMYTFRYGN&url=%2FboAt-Rockerz-550-Headphone-Aesthetics%2Fdp%2FB08R7L77T7%2Fref%3Dsr_1_1_sspa%3Fkeywords%3DHEADPHONES%26qid%3D1653325549%26sr%3D8-1-spons%26psc%3D1&qualifier=1653325549&id=1743891496471419&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A00714251G31W6CBT4LYQ&url=%2FTribit-Bluetooth-Headphones-Lightening-Cancelling%2Fdp%2FB083K349ZR%2Fref%3Dsr_1_2_sspa%3Fkeywords%3DHEADPHONES%26qid%3D1653325549%26smid%3DA10RCQSUCNMNSF%26sr%3D8-2-spons%26psc%3D1&qualifier=1653325549&id=1743891496471419&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/Rockerz-450-Wireless-Bluetooth-Headphone/dp/B07PR1CL3S/ref=sr_1_3?keywords=HEADPHONES&qid=1653325549&sr=8-3
Scraping URL =  https://www.amazon.in/Bassheads-100-Wired-Earphones-Green/dp/B08MD255XB/ref=sr_1_4?keywords=HEADPHONES&qid=16

In [14]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,Brand: boAt,"Boat Rockerz 550 Upto 20 Hours Playback, 50Mm ...",4.1 out of 5 stars,"43,487 ratings",-,7 Days Replacement,-,In stock.,Tap into instant wireless connectivity with th...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Visit the Tribit Store,"Tribit XFree Go Headphones with Mic, Wireless ...",4.2 out of 5 stars,"5,318 ratings",-,7 Days Replacement,"Thursday, May 26",,[Bluetooth 5.2 Wirelss Headphones] Our upgrade...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Brand: boAt,Boat Rockerz 450 Bluetooth Wireless On Ear Hea...,4.1 out of 5 stars,"80,386 ratings",-,7 Days Replacement,-,In stock.,Playback- It provides a massive battery backup...,https://www.amazon.in/Rockerz-450-Wireless-Blu...
3,Brand: boAt,Boat Bassheads 100 Wired in Ear Earphones with...,4.1 out of 5 stars,"323,273 ratings",-,7 Days Replacement,-,Usually dispatched in 4 days.,The perfect way to add some style and stand ou...,https://www.amazon.in/Bassheads-100-Wired-Earp...


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [23]:
driver = webdriver.Chrome(r"C:\Users\Toshiba\chromedriver.exe")
driver = webdriver.Chrome("chromedriver.exe")
time.sleep(2)

In [24]:
driver.get('https://images.google.com/')

In [25]:
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("fruits")       # Inputing "banana" keyword to search rock images

In [26]:
print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")

start scrolling to generate more images on the page...


In [27]:
 images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

In [28]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

152

In [32]:

for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("D:/Dishant/banana"+str(i)+".jpg", "wb")
    file.write(response.content)

# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [ ]:
driver = webdriver.Chrome(r"C:\Users\Toshiba\chromedriver.exe")
driver = webdriver.Chrome("chromedriver.exe")
time.sleep(2)
url4="https://www.flipkart.com/search?q=smartphone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
driver.get(url4)

In [34]:
Brand_Name=[]
Colour=[]
Storage_RAM_ROM=[]
P_F_Camera=[]
Display_size_Resolution=[]
ProcessorAndCores=[]
Battery=[]
Price=[]
Product_URL=[]

In [35]:
#scraping the Brand_Name 
BName=driver.find_elements_by_xpath("//div[@class='_4rR01T']")
for i in BName:
    if i.text is None :
        Brand_Name.append("--") 
    else:
        Brand_Name.append(i.text)
print(len(Brand_Name),Brand_Name)

24 ['POCO C31 (Royal Blue, 64 GB)', 'realme C11 2021 (Cool Blue, 32 GB)', 'realme C11 2021 (Cool Grey, 32 GB)', 'realme C31 (Dark Green, 32 GB)', 'realme C11 2021 (Cool Grey, 64 GB)', 'MOTOROLA G60 (Moonless, 128 GB)', 'realme C11 2021 (Cool Blue, 64 GB)', 'realme C35 (Glowing Black, 64 GB)', 'realme C31 (Light Silver, 32 GB)', 'vivo T1 5G (Rainbow Fantasy, 128 GB)', 'vivo T1 5G (Rainbow Fantasy, 128 GB)', 'vivo T1 5G (Starlight Black, 128 GB)', 'vivo T1 5G (Starlight Black, 128 GB)', 'realme C31 (Dark Green, 64 GB)', 'realme C35 (Glowing Green, 64 GB)', 'REDMI Note 10T 5G (Mint Green, 64 GB)', 'realme C35 (Glowing Black, 128 GB)', 'POCO C31 (Royal Blue, 32 GB)', 'MOTOROLA G60 (Soft Silver, 128 GB)', 'realme C31 (Light Silver, 64 GB)', 'realme Narzo 50A (Oxygen Blue, 64 GB)', 'realme Narzo 50A (Oxygen Green, 64 GB)', 'REDMI Note 10T 5G (Metallic Blue, 64 GB)', 'REDMI 10 (Pacific Blue, 64 GB)']


In [36]:
#scraping the Storage_RAM_ROM 
ram=driver.find_elements_by_xpath("//ul[@class='_1xgFaf']//li[1]")
for i in ram:
    if i.text is None :
        Storage_RAM_ROM.append("--") 
    else:
        Storage_RAM_ROM.append(i.text)
print(len(Storage_RAM_ROM),Storage_RAM_ROM)

24 ['4 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '2 GB RAM | 32 GB ROM | Expandable Upto 256 GB', '2 GB RAM | 32 GB ROM | Expandable Upto 256 GB', '3 GB RAM | 32 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '6 GB RAM | 128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '3 GB RAM | 32 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '3 GB RAM | 32 GB ROM | Expandable Upto 512 GB', '6 GB RAM | 128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB', '4 GB RA

In [37]:
#scraping the P_F_Camera 
PC=driver.find_elements_by_xpath("//ul[@class='_1xgFaf']//li[3]")
for i in PC:
    if i.text is None :
        P_F_Camera.append("--") 
    else:
        P_F_Camera.append(i.text)
print(len(P_F_Camera),P_F_Camera)

24 ['13MP + 2MP + 2MP | 5MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '13MP + 2MP + 0.3MP | 5MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '108MP + 8MP + 2MP | 32MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '50MP + 2MP + 0.3MP | 8MP Front Camera', '13MP + 2MP + 0.3MP | 5MP Front Camera', '50MP + 2MP + 2MP | 16MP Front Camera', '50MP + 2MP + 2MP | 16MP Front Camera', '50MP + 2MP + 2MP | 16MP Front Camera', '50MP + 2MP + 2MP | 16MP Front Camera', '13MP + 2MP + 0.3MP | 5MP Front Camera', '50MP + 2MP + 0.3MP | 8MP Front Camera', '48MP Primary Camera + 2MP Macro Lens + 2MP Depth Sensor | 8MP Front Camera', '50MP + 2MP + 0.3MP | 8MP Front Camera', '13MP + 2MP + 2MP | 5MP Front Camera', '108MP + 8MP + 2MP | 32MP Front Camera', '13MP + 2MP + 0.3MP | 5MP Front Camera', '50MP + 2MP + 2MP | 8MP Front Camera', '50MP + 2MP + 2MP | 8MP Front Camera', '48MP Primary Camera + 2MP Macro Lens + 2MP Depth Sensor | 8MP Front Camera', '50

In [38]:
#scraping the Display_size_Resolution 
DS=driver.find_elements_by_xpath("//ul[@class='_1xgFaf']//li[2]")
for i in DS:
    if i.text is None :
        Display_size_Resolution.append("--") 
    else:
        Display_size_Resolution.append(i.text)
print(len(Display_size_Resolution),Display_size_Resolution)

24 ['16.59 cm (6.53 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.56 cm (6.52 inch) HD Display', '16.51 cm (6.5 inch) HD+ Display', '17.22 cm (6.78 inch) Full HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.76 cm (6.6 inch) Full HD+ Display', '16.56 cm (6.52 inch) HD Display', '16.71 cm (6.58 inch) Full HD+ Display', '16.71 cm (6.58 inch) Full HD+ Display', '16.71 cm (6.58 inch) Full HD+ Display', '16.71 cm (6.58 inch) Full HD+ Display', '16.56 cm (6.52 inch) HD Display', '16.76 cm (6.6 inch) Full HD+ Display', '16.66 cm (6.56 inch) Full HD+ Display', '16.76 cm (6.6 inch) Full HD+ Display', '16.59 cm (6.53 inch) HD+ Display', '17.22 cm (6.78 inch) Full HD+ Display', '16.56 cm (6.52 inch) HD Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.66 cm (6.56 inch) Full HD+ Display', '17.02 cm (6.7 inch) HD+ Display']


In [39]:
#scraping the ProcessorAndCores 
P=driver.find_elements_by_xpath("//ul[@class='_1xgFaf']//li[5]")
for i in P:
    if i.text is None :
        ProcessorAndCores.append("--") 
    else:
        ProcessorAndCores.append(i.text)
print(len(ProcessorAndCores),ProcessorAndCores)

24 ['MediaTek Helio G35 Processor', 'Octa-core Processor', 'Octa-core Processor', 'Unisoc T612 Processor', 'Octa-core Processor', 'Qualcomm Snapdragon 732G Processor', 'Octa-core Processor', 'Unisoc Tiger T616 Processor', 'Unisoc T612 Processor', 'Qualcomm Snapdragon 695 Processor', 'Qualcomm Snapdragon 695 Processor', 'Qualcomm Snapdragon 695 Processor', 'Qualcomm Snapdragon 695 Processor', 'Unisoc T612 Processor', 'Unisoc Tiger T616 Processor', 'Mediatek Dimensity 700 Processor', 'Unisoc Tiger T616 Processor', 'MediaTek Helio G35 Processor', 'Qualcomm Snapdragon 732G Processor', 'Unisoc T612 Processor', 'MediaTek Helio G85 Processor', 'MediaTek Helio G85 Processor', 'Mediatek Dimensity 700 Processor', 'Qualcomm Snapdragon 680 Processor']


In [40]:

#scraping the Battery 
B=driver.find_elements_by_xpath("//ul[@class='_1xgFaf']//li[4]")
for i in B:
    if i.text is None :
        Battery.append("--") 
    else:
        Battery.append(i.text)
print(len(Battery),Battery)

24 ['5000 mAh Lithium-ion Polymer Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '6000 mAh Battery', '5000 mAh Battery', '5000 mAh Lithium Polymer Battery', '5000 mAh Battery', '5000 mAh Lithium Battery', '5000 mAh Lithium Battery', '5000 mAh Lithium Battery', '5000 mAh Lithium Battery', '5000 mAh Battery', '5000 mAh Lithium Polymer Battery', '5000 mAh Lithium Polymer Battery', '5000 mAh Lithium Polymer Battery', '5000 mAh Lithium-ion Polymer Battery', '6000 mAh Battery', '5000 mAh Battery', '6000 mAh Battery', '6000 mAh Battery', '5000 mAh Lithium Polymer Battery', '6000 mAh Lithium Polymer Battery']


In [41]:
#scraping the Price 
price=driver.find_elements_by_xpath("//div[@class='_30jeq3 _1_WHN1']")
for i in price:
    if i.text is None :
        Price.append("--") 
    else:
        Price.append(i.text)
print(len(Price),Price)

24 ['₹9,499', '₹7,499', '₹7,499', '₹9,299', '₹8,999', '₹15,999', '₹8,999', '₹11,999', '₹9,299', '₹15,990', '₹16,990', '₹15,990', '₹16,990', '₹9,999', '₹11,999', '₹11,999', '₹12,999', '₹8,499', '₹15,999', '₹9,999', '₹11,649', '₹11,649', '₹11,999', '₹10,499']


In [42]:
FlipKart=pd.DataFrame([])
FlipKart['Brand_Name']=Brand_Name
FlipKart['Storage_RAM_ROM']=Storage_RAM_ROM
FlipKart['Amount P_F_Camera']=P_F_Camera
FlipKart['Display_size_Resolution']=Display_size_Resolution
FlipKart['ProcessorAndCores']=ProcessorAndCores
FlipKart['Battery']=Battery
FlipKart['Price']=Price

FlipKart

,Brand_Name,Storage_RAM_ROM,Amount P_F_Camera,Display_size_Resolution,ProcessorAndCores,Battery,Price
0,"POCO C31 (Royal Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP + 2MP + 2MP | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,MediaTek Helio G35 Processor,5000 mAh Lithium-ion Polymer Battery,"₹9,499"
1,"realme C11 2021 (Cool Blue, 32 GB)",2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,8MP Rear Camera | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Octa-core Processor,5000 mAh Battery,"₹7,499"
2,"realme C11 2021 (Cool Grey, 32 GB)",2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,8MP Rear Camera | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Octa-core Processor,5000 mAh Battery,"₹7,499"
3,"realme C31 (Dark Green, 32 GB)",3 GB RAM | 32 GB ROM | Expandable Upto 1 TB,13MP + 2MP + 0.3MP | 5MP Front Camera,16.56 cm (6.52 inch) HD Display,Unisoc T612 Processor,5000 mAh Battery,"₹9,299"
4,"realme C11 2021 (Cool Grey, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,8MP Rear Camera | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Octa-core Processor,5000 mAh Battery,"₹8,999"
5,"MOTOROLA G60 (Moonless, 128 GB)",6 GB RAM | 128 GB ROM,108MP + 8MP + 2MP | 32MP Front Camera,17.22 cm (6.78 inch) Full HD+ Display,Qualcomm Snapdragon 732G Processor,6000 mAh Battery,"₹15,999"
6,"realme C11 2021 (Cool Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,8MP Rear Camera | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Octa-core Processor,5000 mAh Battery,"₹8,999"
7,"realme C35 (Glowing Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,50MP + 2MP + 0.3MP | 8MP Front Camera,16.76 cm (6.6 inch) Full HD+ Display,Unisoc Tiger T616 Processor,5000 mAh Lithium Polymer Battery,"₹11,999"
8,"realme C31 (Light Silver, 32 GB)",3 GB RAM | 32 GB ROM | Expandable Upto 1 TB,13MP + 2MP + 0.3MP | 5MP Front Camera,16.56 cm (6.52 inch) HD Display,Unisoc T612 Processor,5000 mAh Battery,"₹9,299"
9,"vivo T1 5G (Rainbow Fantasy, 128 GB)",4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP + 2MP + 2MP | 16MP Front Camera,16.71 cm (6.58 inch) Full HD+ Display,Qualcomm Snapdragon 695 Processor,5000 mAh Lithium Battery,"₹15,990"


# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.


In [43]:
driver = webdriver.Chrome(r"C:\Users\Toshiba\chromedriver.exe")
driver = webdriver.Chrome("chromedriver.exe")
time.sleep(2)

In [44]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)


In [45]:
city = input('Enter City Name : ')                                         # Enter city to be searched"

Enter City Name : Nagpur


In [46]:
search = driver.find_element_by_id("searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element_by_id("searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

In [47]:
try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

URL Extracted:  https://www.google.co.in/maps/place/Nagpur,+Maharashtra/@21.1613484,78.9324201,11z/data=!4m5!3m4!1s0x3bd4c0a5a31faf13:0x19b37d06d0bb3e2b!8m2!3d21.1458004!4d79.0881546
Latitude = 21.1613484, Longitude = 78.9324201


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in

In [48]:
driver = webdriver.Chrome(r"C:\Users\Toshiba\chromedriver.exe")
driver = webdriver.Chrome("chromedriver.exe")
time.sleep(2)

In [49]:

url1="https://trak.in/india-startup-funding-investment-2015/"

In [50]:

driver.get(url1)

In [51]:
Dates=[]
Company=[]
Industry=[]
Investor_Name=[]
Investment_Type=[]
Amount=[]

In [52]:
#scraping the company_name 
companies=driver.find_elements_by_xpath("//td[@class='column-3']")
for i in companies:
    if i.text is None :
        Company.append("--") 
    else:
        Company.append(i.text)
print(len(Company),Company)

392 ['BYJU’S', 'Meesho', 'Swiggy', 'Groww', 'Beldara', 'DealShare', 'Uniphore', 'Dunzo', 'BYJU’S', 'SkilloVilla', 'CityMall', 'DotPe', 'Doubtnut', 'Zomato', 'Fingerlix', 'Zolve', 'KreditBee', 'Pepperfry', 'Grofers', 'Nothing', 'SplashLearn', 'Digit Insurance', 'Bombay Shaving Company', 'DeHaat', 'Darwinbox', 'mfine', 'Udayy', 'True Elements', 'Saveo', 'Bira 91', 'Pine Labs', 'Zenoti', 'Tax Buddy India', 'Delhivery', 'PagarBook', 'Codingal', 'CRED', 'Bira 91', 'Chumbak', 'Cityflo', 'Shipsy', 'Unacademy', 'Cashfree', 'Credgenics', 'Country Delight', 'Nykaa', 'Betterplace', 'FlexiLoans', 'GetVantage', 'FreshToHome', 'Origo', 'Origo', 'Treebo Hotels', 'Hubilo', 'CredAble', 'Byju’s', 'mCaffeine', 'Qshala', 'Winzo', 'Hippo Video', 'Melorra', '1mg', 'mfine', 'Apna', 'Railofy', 'Practo', 'Medlife', 'HungerBox', 'Dunzo', 'Terra.do', 'Classplus', 'Niyo', 'ZestMoney', 'FreshToHome', 'Eduvanz', 'Flipkart', 'Vedantu', 'Crio', 'goDutch', 'Mystifly', 'JetSynthesys', 'gigIndia', 'PumPumPum', 'FLYX', '

In [53]:

#scraping the Industry 
Ind=driver.find_elements_by_xpath("//td[@class='column-4']")
for i in Ind:
    if i.text is None :
        Industry.append("--") 
    else:
        Industry.append(i.text)
print(len(Industry),Industry)

392 ['Edu-tech', 'E-commerce', 'Online Food Delivery', 'FinTech', 'E-commerce', 'E-commerce', 'Technology', 'E-commerce', 'Edu-tech', 'Edu-tech', 'E-commerce', 'FinTech', 'Edu Tech', 'Hospitality', 'Hospitality', 'FinTech', 'Finance', 'E-commerce', 'E-Commerce', 'Technology', 'EdTech', 'Financial Services', 'Consumer Goods Company', 'AgriTech Startup', 'SaaS', 'Health Tech Startup', 'EdTech', 'Food Startup', 'B2B E-commerce', 'Food and Beverage', 'Financial Services', 'Technology Provider', 'FinTech', 'Supply chain', 'SaaS', 'EduTech', 'Financial Services', 'Food and Beverage', 'Lifestyle', 'Transportation', 'SaaS', 'Edutech', 'FinTech', 'Fintech', 'Dairy Tech', 'E-commerce', 'Digital Solutions', 'FinTech', 'FinTech', 'E-commerce', 'AgriTech', 'Advertising', 'Hospitality', 'Information Technology', 'FinTech', 'EduTech', 'Personal Care', 'EduTech', 'Online Gaming', 'Video Customer Experience(CX) Platform', 'E-commerce', 'E-commerce', 'HealthTech', 'Human Resources', 'Transportation', 'H

In [54]:
#scraping the Dates 
dt=driver.find_elements_by_xpath("//td[@class='column-2']")
for i in dt:
    if i.text is None :
        Dates.append("--") 
    else:
        Dates.append(i.text)
print(len(Dates),Dates)

392 ['01/04/2021', '05/04/2021', '14/04/2021', '07/04/2021', '14/04/2021', '04/03/2021', '31/03/2021', '30/03/2021', '30/03/2021', '23/03/2021', '25/03/2021', '26/03/2021', '11/02/2021', '22/02/2021', '19/02/2021', '17/02/2021', '15/02/2021', '12/02/2021', '12/02/2021', '09/02/2021', '09/02/2021', '15/01/2021', '28/01/2021', '19/01/2021', '19/01/2021', '18/01/2021', '18/01/2021', '11/01/2021', '13/01/2021', '15/12/2020', '21/12/2020', '15/12/2020', '22/12/2020', '15/12/2020', '18/12/2020', '30/11/2020', '30/11/2020', '29/11/2020', '27/11/2020', '25/11/2020', '25/11/2020', '25/11/2020', '24/11/2020', '30/11/2020', '23/11/2020', '28/10/2020', '28/10/2020', '28/10/2020', '27/10/2020', '27/10/2020', '27/10/2020', '27/10/2020', '15/10/2020', '26/10/2020', '28/10/2020', '08/09/2020', '12/09/2020', '09/09/2020', '02/09/2020', '09/09/2020', '07/09/2020', '07/09/2020', '31/08/2020', '31/08/2020', '03/09/2020', '15/08/2020', '13/08/2020', '13/08/2020', '04/08/2020', '11/08/2020', '12/08/2020', '

In [55]:
#scraping the Investor_Name 
IN=driver.find_elements_by_xpath("//td[@class='column-7']")
for i in IN:
    if i.text is None :
        Investor_Name.append("--") 
    else:
        Investor_Name.append(i.text)
print(len(Investor_Name),Investor_Name)

392 ['Innoven Capital', 'SoftBank Vision Fund 2', 'Amansa Holdings, Carmignac, Falcon Edge Capital, Goldman Sachs, Think Investments', 'MC Global Edtech, B Capital, Baron, others', 'Hindustan Media Ventures', 'Innoven Capital', 'Sorenson Capital Partners', 'Krishtal Advisors Pte Ltd', 'MC Global Edtech, B Capital, Baron, others', 'Titan Capital, others', 'Accel Partners', 'PayU', 'SIG Global, Sequoia Capital, WaterBridge Ventures and ON Mauritius', 'Tiger Global, Kora', 'Rhodium Trust, Accel Partners and Swiggy', 'Accel Partners and Lightspeed Venture Partners', 'Azim Premji’s PremjiInvest and South Korea’s Mirae Asset Venture', 'InnoVen Capital', 'SoftBank Vision Fund (SVF)', 'GV', 'Owl Ventures', 'A91 Partners, Faering Capital, TVS Capital Funds', 'Reckitt Benckiser', 'Prosus Ventures', 'Salesforce Ventures', 'Heritas Capital Management', 'Sequoia Capital', 'SIDBI Venture Capital', 'Matrix Partners India, RTP Global, others', 'Nishant Mittal, Pankaj Chaddah and Shikha Sunil Chandak',

In [56]:
#scraping the Investment_Type 
IT=driver.find_elements_by_xpath("//td[@class='column-8']")
for i in IT:
    if i.text is None :
        Investment_Type.append("--") 
    else:
        Investment_Type.append(i.text)
print(len(Investment_Type),Investment_Type)

392 ['Series F', 'Series E', 'Series J', 'Series D', 'Venture', 'Debt Financing', 'Series D', 'Series E', 'Series F', 'Seed', 'Series A', 'Series A', 'Series B', 'Venture', 'Series C', 'Seed', 'Series C', 'Debt Financing', 'Unspecified', 'Series A', 'Series C', 'Venture', 'Venture', 'Series C', 'Seed', 'Venture Round', 'Seed Funding', 'Series', 'Seed', 'Funding', 'Secondary Market', 'Series D', 'Seed', 'Secondary Market', 'Series A', 'Seed', 'Series C', 'Bridge Funding', 'Pre-series E', 'Series A', 'Series A', 'Venture', 'Series B', 'Pre-Series A', 'Series C', 'Personal Investment', 'Series B', 'Equity and Debt', 'Seed', 'Series C', 'Debt', 'Series A', '', 'Seed', 'Follow-On', 'Private Equity', 'Series B', 'Angel', 'Series B', 'Series A', 'Debt Financing', 'In Progress', 'Series B', 'Series A', 'Seed', 'Series F', '', 'Series D1', 'In Progress', 'Seed', 'In Progress', '', '', 'Venture', 'Series A', 'M&A', 'Series D', 'pre-Series A', 'Seed', 'pre-Series B', 'Venture-Series Unknown', 'pr

In [57]:
#scraping the Amount 
Price=driver.find_elements_by_xpath("//td[@class='column-9']")
for i in Price:
    if i.text is None :
        Amount.append("--") 
    else:
        Amount.append(i.text)
print(len(Amount),Amount)

392 ['460,000,000', '300,000,000', '343,000,000', '83,000,000', '7,400,000', '250,000,000', '140,000,000', '8,000,000', '460,000,000', '300,000,000', '11,000,000', '27,500,000', '2,500,000', '250,000,000', '2,747,045.20', '1,50,00,000', '75,000,000', '4,773,958', '55,000,000', '15,000,000', '18,000,000', '1,80,00,000', '6,172,258.50', '30,000,000', '15,000,000', '16,000,000', '15,000,000', '100,000,000', '4,000,000', '1,273,634.07', 'Undisclosed', '160,000,000', '1,000,000', '25,000,000', '15,000,000', '560,000', '80,000,000', '3,000,000', '1,001,047.92', '7,700,000', '6,003,806', '75,000,000-100,000,000', '35,300,000', '3,500,000', '25,000,000', 'NA', '10,000,000', '20,160,000', '5,000,000', '121,000,000', '4,700,000', '15,000,000', '6,000,000', '4,500,000', '4,500,000', '500,000,000', '3,000,000', '370,000', '15,500,000', '4,500,000', 'upto 8,900,000', '100,000,000', '5,400,000', '8,000,000', '950,000', '32,000,000', '23,000,000', '1,560,000', '30,000,000', '1,400,000', 'upto 15,000,

In [58]:
Funding=pd.DataFrame([])
Funding['Company']=Company
Funding['Industry']=Industry
Funding['Investor_Name']=Investor_Name
Funding['Amount Invested']=Amount
Funding['Specification']=Investment_Type
Funding['Dates']=Dates
Funding

,Company,Industry,Investor_Name,Amount Invested,Specification,Dates
0,BYJU’S,Edu-tech,Innoven Capital,"460,000,000",Series F,01/04/2021
1,Meesho,E-commerce,SoftBank Vision Fund 2,"300,000,000",Series E,05/04/2021
2,Swiggy,Online Food Delivery,"Amansa Holdings, Carmignac, Falcon Edge Capita...","343,000,000",Series J,14/04/2021
3,Groww,FinTech,"MC Global Edtech, B Capital, Baron, others","83,000,000",Series D,07/04/2021
4,Beldara,E-commerce,Hindustan Media Ventures,"7,400,000",Venture,14/04/2021
...,...,...,...,...,...,...
387,zippserv,Consumer Internet,Info Edge (India) Ltd,"440,000",Seed/ Angel Funding,09/07/2018
388,Groww,Consumer Internet,"Insignia Ventures Partners, Lightbridge Partne...","1,100,000",Seed/ Angel Funding,09/07/2018
389,Avenue Growth,Consumer Internet,"Avtar Monga, chief operating office at IDFC Ba...","437,000",Seed/ Angel Funding,10/07/2018
390,iNICU,Healthcare,Venture Catalysts,N/A,Seed/ Angel Funding,10/07/2018


# 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [59]:
driver = webdriver.Chrome(r"C:\Users\Toshiba\chromedriver.exe")
driver = webdriver.Chrome("chromedriver.exe")
time.sleep(2)

In [60]:

url="https://www.digit.in/top-products/best-gaming-laptops-40.html"

In [61]:

driver.get(url)

In [62]:
Brands=[]
Products_Description=[]
Specification=[]
Price=[]

In [63]:
br=driver.find_elements_by_xpath("//div[@class='TopNumbeHeading active sticky-footer']")
len(br)

10

In [64]:
for i in br:
   
    Brands.append(str(i.text).replace("\n",""))
Brands

['1.MSI RAIDER GE76',
 '2.ASUS ROG STRIX SCAR 15',
 '3.ACER NITRO 5',
 '4.MSI STEALTH 15M',
 '5.ASUS ROG STRIX SCAR 15',
 '6.ASUS ROG STRIX SCAR 15',
 '7.ASUS ZEPHYRUS G14',
 '8.HP OMEN 16',
 '9.ASUS ROG ZEPHYRUS DUO 15',
 '10.ACER ASPIRE 7 GAMING LAPTOP']

In [65]:
sp=driver.find_elements_by_xpath("//div[@class='Specs-Wrap']")
len(sp)

10

In [66]:
for i in sp:
   
    Specification.append(str(i.text).replace("\n",""))
Specification

['OSWINDOWS 11 HOMEDISPLAY17" (3840 X 2160)PROCESSOR12TH GEN INTEL CORE I9-12900HK | 5 GHZMEMORY2 TB SSD/16 GBGB DDR5',
 'OSWINDOWS 11 HOMEDISPLAY15.6" (2560 X 1440)PROCESSOR12TH GEN INTEL CORE I9-12900H | 2.5 GHZMEMORY2 TB SSD/32 GBGB DDR5',
 'OSWINDOWS 10DISPLAY15.6" (1920 X 1080)PROCESSORAMD RYZEN 9 OCTA CORE | 2.4 GHZMEMORY1 TB HDD/16 GBGB DDR4',
 'OSWINDOWS 10DISPLAY15.6" (1920 X 1080)PROCESSORINTEL CORE I7 11TH GEN - 11375H | NAMEMORY1 TB SSD/16 GBGB DDR4',
 'OSWINDOWS 10DISPLAY15.6" (2560 X 1440)PROCESSORAMD RYZEN 9 OCTA CORE - 5900HX | 3.3 GHZMEMORY2 TB SSD/32 GBGB DDR4',
 'OSWINDOWS 10 HOMEDISPLAY15.6" (1920 X 1080)PROCESSORAMD RYZEN™ 9 5900HX | 3.3 GHZMEMORY1 TB SSD/16 GBGB DDR4',
 'OSWINDOWS 10 HOMEDISPLAY14" (1920 X 1080)PROCESSORAMD 3RD GEN RYZEN 9 | 3.3 GHZMEMORY1 TB SSD/16 GBGB DDR4',
 'OSWINDOWS 11 HOMEDISPLAY16.1" (1920 X 1080)PROCESSOR12TH GEN INTEL CORE I7-12700H | 4.7 GHZMEMORY1 TB SSD/16 GBGB DDR4',
 'OSWINDOWS 10DISPLAY15.6" (3840 X 1100)PROCESSORINTEL CORE I7 10T

In [67]:
des=driver.find_elements_by_xpath("//div[@class='Section-center']")
len(des)

10

In [68]:
for i in des:
   
    Products_Description.append(str(i.text).replace("\n",""))
Products_Description

['Possibly the most high-end gaming laptop anyone can buy is the MSI Raider GTE76 2022 edition. Last year’s model was impressive that it won the Digit Zero 1 award for the best gaming laptop bar none. The current year model is no different either. It comes with an Intel 12th Gen Core i9-12900HK chip, which is a 14-core, 20-thread CPU. It is probably the most powerful processor in a laptop at the time of writing this article. It is paired with an equally capable Nvidia GeForce RTX 3080 Ti with 16GB GDDR6 VRAM featuring 175W TGP. The laptop also features 32GB DDR5 memory running in Quad-channel mode. To top it all off, the MSI Raider GE76 also comes equipped with a 2TB NVMe PCIe Gen 4 SSD to load all your files and games in an instant. If you have the budget, then the MSI Raider GE76 stands top of the list of the best gaming laptops and it’ll do so probably for a while now.SPECIFICATIONOS : Windows 11 HomeDisplay : 17" (3840 x 2160)Processor : 12th Gen Intel Core i9-12900HK | 5 GHzMemory

In [69]:
pri=driver.find_elements_by_xpath("//td[@class='smprice']")
len(pri)

10

In [70]:
for i in pri:
   
    Price.append(str(i.text).replace("\n",""))
Price

['₹ 429,940',
 '₹ 285,390',
 '₹ 129,990',
 '₹ 134,990',
 '₹ 193,990',
 '₹ 215,990',
 '₹ 144,990',
 '₹ 139,990',
 '₹ 185,000',
 '₹ 53,490']

In [71]:

digit_lap=pd.DataFrame([])
digit_lap['Brands']=Brands[0:10]
digit_lap['Price']=Price[0:10]
digit_lap['Specification']=Specification[0:10]
digit_lap['Description']=Products_Description[0:10]
digit_lap

,Brands,Price,Specification,Description
0,1.MSI RAIDER GE76,"₹ 429,940","OSWINDOWS 11 HOMEDISPLAY17"" (3840 X 2160)PROCE...",Possibly the most high-end gaming laptop anyon...
1,2.ASUS ROG STRIX SCAR 15,"₹ 285,390","OSWINDOWS 11 HOMEDISPLAY15.6"" (2560 X 1440)PRO...",The second high-end gaming laptop on the list ...
2,3.ACER NITRO 5,"₹ 129,990","OSWINDOWS 10DISPLAY15.6"" (1920 X 1080)PROCESSO...",Possibly the best value-for-money gaming lapto...
3,4.MSI STEALTH 15M,"₹ 134,990","OSWINDOWS 10DISPLAY15.6"" (1920 X 1080)PROCESSO...",If you’re looking for a powerful gaming laptop...
4,5.ASUS ROG STRIX SCAR 15,"₹ 193,990","OSWINDOWS 10DISPLAY15.6"" (2560 X 1440)PROCESSO...",If you want possibly the best performance poss...
5,6.ASUS ROG STRIX SCAR 15,"₹ 215,990","OSWINDOWS 10 HOMEDISPLAY15.6"" (1920 X 1080)PRO...",When the ASUS ROG Strix Scar 15 ended up on ou...
6,7.ASUS ZEPHYRUS G14,"₹ 144,990","OSWINDOWS 10 HOMEDISPLAY14"" (1920 X 1080)PROCE...",The Asus Zephyrus G14 is a first-of-its-kind g...
7,8.HP OMEN 16,"₹ 139,990","OSWINDOWS 11 HOMEDISPLAY16.1"" (1920 X 1080)PRO...",It is difficult to find a laptop that offers a...
8,9.ASUS ROG ZEPHYRUS DUO 15,"₹ 185,000","OSWINDOWS 10DISPLAY15.6"" (3840 X 1100)PROCESSO...",The machine is powered by an Intel Core i7-108...
9,10.ACER ASPIRE 7 GAMING LAPTOP,"₹ 53,490","OSWINDOWS 10 HOMEDISPLAY15.6"" (1920 X 1080)PRO...",The Acer Aspire 7 gaming laptop is a very capa...
